In [2]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import os,sys
import numpy as np
import nltk

In [3]:
owd=os.getcwd()

In [4]:
def Tokenizer(f):
    f=f.lower()
    tokens=nltk.word_tokenize(f) #tokenize
    
    stop = set(stopwords.words('english')) #clear stop words
    tokens=[i for i in tokens if i not in stop]
    
    porter_stemmer = PorterStemmer() #stemming
    tokens=[porter_stemmer.stem(i) for i in tokens]
    
    return tokens

In [5]:
def normal_indexing(filenames):
    
    normal_Index = {} #indexing tokens using file name
    
    for file in filenames:
        fo = open(file,'rt',encoding='UTF8')
        data=fo.read()
        fo.close()
        tokens=Tokenizer(data)
        location = {} #locaton of tokens in a file
        
        for index, token in enumerate(tokens):
            
            if token in location.keys():
                location[token].append(index)
            else:
                location[token]=[index]
        
        normal_Index[file]=location
    
    return normal_Index

**Inverted Index**

In [6]:
def inverted_indexing(normal_Index):
    inverted_Index={} #inverted index for computationg tf-idf :0
    
    for file in normal_Index.keys():
        for token in normal_Index[file].keys():
            if token in inverted_Index.keys():
                if token in inverted_Index[token].keys():
                    inverted_Index[token][file].extend(normal_Index[file][token][:])
                else:
                    inverted_Index[token][file]=normal_Index[file][token]
            else:
                inverted_Index[token]={file: normal_Index[file][token]}
    return inverted_Index

**Tf-idf is a weighting scheme that assigns each term in a document a weight based on its term frequency (tf) and inverse document frequency (idf).  The terms with higher weight scores are considered to be more important. It’s one of the most popular weighting schemes in Information Retrieval.**
*reference: http://www.ardendertat.com/2011/07/17/how-to-implement-a-search-engine-part-3-ranking-tf-idf/*

In [7]:
def Query(fq):
    fo = open(fq,'rt',encoding='UTF8')
    data=fo.read()
    fo.close()
    query=Tokenizer(data)
    return query

In [8]:
def filtered_by_query(query, inverted_index):
    
    setlist=[]  
    pre_matrix={}
    f_inverted_index={}
     
    for token in inverted_index.keys():
        if token in query:
            f_inverted_index[token]=inverted_index[token]
    
    for token in f_inverted_index.keys():
        filelist=f_inverted_index[token].keys()
        temp=set(filelist)
        setlist.append(temp)
    
    common_files=setlist[0]    
    
    for i in range(len(setlist)):
        if common_files!= setlist[i]:
            common_files = common_files.intersection(setlist[i])
   
    f_file_list=[file for file in common_files]
    
    for token in inverted_index.keys():
        for file in inverted_index[token].keys():
            if file in f_file_list:
                location=inverted_index[token][file]
                
                if file in pre_matrix:
                    pre_matrix[file][token]=len(location)
                else:
                    pre_matrix[file]={token:len(location)}
    
    return pre_matrix, f_inverted_index

In [9]:
def tf_idf(pre_matrix, inverted_index, filenames, query):
    
    setlist=[]
    fi_total_vec={}
    total_words=set([])
    
    N=len(filenames)
    
    for file in pre_matrix.keys():
        total_words.update(pre_matrix[file].keys())
        
    word_index=[word for word in total_words]
    word_index.sort()
    
    for file in pre_matrix.keys():
        token_list=pre_matrix[file].keys()
        doc_vec=[]
        for word in word_index:
            if word in token_list:
                df=len(inverted_index[word].keys())
                tf=pre_matrix[file][word]
                tf_idf=np.log(1+tf)*np.log10(N/df)
                doc_vec.append(tf_idf)
            else:
                doc_vec.append(0)
                
        fi_total_vec[file]=doc_vec
        
    q_tf=len(query)
    query_vec=[]
    
    for word in word_index:
        if word in query:
            query_vec.append(1)
        else:
            query_vec.append(0)
    
    for word in query:
        q_df=len(inverted_index[word].keys())
        tf_idf=np.log(1+q_tf)*np.log10(N/q_df)
        index = word_index.index(word)
        query_vec[index]=query_vec[index]*tf_idf
        
    return word_index, fi_total_vec, query_vec

In [9]:
'''def query_vecterization(query, word_index):
    
    q_tf=len(query)
    query_vec=[]
    
    for word in word_index:
        if word in query:
            query_vec.append(1)
        else:
            query_vec.append(0)
            
    for vec in query_vec:
        
    return query_vec'''

In [10]:
def tf_idf_score(query_vec, fi_total_vec):
    
    tf_idf_score={}
    
    for file in fi_total_vec.keys():
        score=[]
        for i in range(len(query_vec)):
            if query_vec[i]:
                score.append(fi_total_vec[file][i])
        tf_idf_score[file]=np.sum(score)
       
    return tf_idf_score

** cos similarity**

In [11]:
def cos_similarity(query_vec, fi_total_vec):
    
    cos_sim_score={}
    
    x=np.array(query_vec)
    normalizing_factor_x = np.sqrt(np.sum(np.square(x)))
    
    for file in fi_total_vec.keys():
        y=np.array(fi_total_vec[file])
        normalizing_factor_y = np.sqrt(np.sum(np.square(y)))
        cos_sim_score[file]=np.matmul(x,np.transpose(y))/(normalizing_factor_x*normalizing_factor_y)
    
    return cos_sim_score

** Rank **

In [12]:
def ranking(score):
    rank_by_score={}
    rank_by_score=sorted(score, key=score.get, reverse=True)[:5]
    return rank_by_score

In [13]:
os.chdir('data')
filenames=os.listdir()
normal_index=normal_indexing(filenames)
os.chdir(owd)

inverted_index=inverted_indexing(normal_index)
query=Query('query.txt')
pre_matrix, f_inverted_index=filtered_by_query(query, inverted_index)
word_index, fi_total_vec, query_vec=tf_idf(pre_matrix, inverted_index, filenames,query)

tf_idf_score=tf_idf_score(query_vec, fi_total_vec)
cos_score=cos_similarity(query_vec, fi_total_vec)
final_rank = ranking(cos_score)

#1. result of running inverted index
print(f_inverted_index)

#2. result of running Tf-idf
print(fi_total_vec)

#3. ranking
print(final_rank)

for file in final_rank:
    print(cos_score[file])


{'presid': {'14.txt': [5], '17.txt': [316, 323], '30.txt': [45], '36.txt': [304], '40.txt': [37, 125], '41.txt': [52, 76, 146, 206, 288, 432, 517, 690, 744], '45.txt': [51], '46.txt': [4, 84], '47.txt': [3, 74, 231, 296], '48.txt': [153, 362], '49.txt': [19, 73, 85, 147, 172, 248, 477, 561], '50.txt': [263, 388], '51.txt': [86, 154, 193, 269, 304, 318, 614, 1007], '52.txt': [6, 176, 694], '53.txt': [49, 421], '54.txt': [14, 20, 36, 69, 110, 130, 133, 157, 180], '55.txt': [0], '56.txt': [113, 199], '57.txt': [14, 151, 377, 744], '58.txt': [15, 32, 53, 74, 135, 154, 186, 227], '59.txt': [5, 147], '6.txt': [502], '7.txt': [120], '9.txt': [569]}, 'obama': {'36.txt': [305], '40.txt': [52], '41.txt': [269, 428], '43.txt': [259, 307], '44.txt': [164, 197, 686, 730, 784, 963, 981], '46.txt': [6, 33, 54, 80, 92, 131, 278, 367], '47.txt': [5, 23, 55, 266, 465, 564, 727, 772, 894, 974], '48.txt': [155, 175, 272, 342], '49.txt': [309, 371], '50.txt': [141, 148, 336, 383, 415, 479], '53.txt': [20, 